In [ ]:
# Dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from dash import Dash, html, dash_table, dcc, Input, Output, State
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
import dash_leaflet as dl


In [ ]:
# set path
original_data = "Australian_Shark_Incident_Database_Public_Version.csv"

# read the file
original_df = pd.read_csv(original_data, encoding = "CP1252")
original_df

In [ ]:
# keep relevant columns
original_df = original_df[['Incident.year',
                           'Victim.injury',
                           'State',
                           'Latitude',
                           'Longitude',
                           'Shark.common.name',
                           'Shark.scientific.name',
                          'Provoked/unprovoked',
                          'Site.category',
                          'Incident.month']]
original_df

In [ ]:
# replace the nulls with unidentified
clean_df = original_df.fillna('unknown')

In [ ]:
yearly_df = clean_df.loc[:,["Incident.year", "State"]]
yearly_df

In [ ]:
states = yearly_df['State'].unique()
states

In [ ]:
yearly_count = yearly_df['Incident.year'].value_counts()
yearly_count

In [ ]:
yearly_df = yearly_df[['Incident.year', 'State']].value_counts().reset_index(name='Count')
yearly_df

In [ ]:
yearly_df.loc[yearly_df['State'] == 'NSW'].sort_values(by=['Incident.year'])

In [ ]:
yearly_grouped = yearly_df.groupby(['Incident.year']).sum().reset_index()
yearly_grouped

In [ ]:
# select my chosen columns focusing on shark types based on state
reduced_df = clean_df.loc[:, ["Shark.common.name", "State"]]
reduced_df

In [ ]:
# an overall total count by shark common name
total_name = reduced_df['Shark.common.name'].value_counts()
total_name

In [ ]:
# count by shark name for each state
totals_df = reduced_df[['State', 'Shark.common.name']].value_counts().reset_index(name='count')
totals_df.head(20)

In [ ]:
totals_df.groupby(['State']).value_counts().head(20)

In [ ]:
dff = totals_df
dff

In [ ]:
# Get unique states and shark common names for dropdown options
states = dff['State'].unique()
names = dff['Shark.common.name'].unique()

In [21]:
# Initialize the app
app = Dash(__name__)

In [22]:
# Define app layout
dropdown = dcc.Dropdown(id='dropdown', options=[{'label': state, 'value': state} for state in yearly_df['State'].unique()])
line_graph = dcc.Graph(id='line_graph')

app.layout = html.Div([
    html.H1("Shark Attack Data"),
    html.H4("Select State"),
    html.Div([
        html.H1(""),
        # Dropdown menu for selecting states
        dcc.Dropdown(
            id='state-dropdown',
            options=[{'label': state, 'value': state} for state in states],
            value='NSW',  # Default value
            multi=False,  # Allow single selection
            style={'width': '50%'}
        ),
        # Graph component for displaying the bar chart
        dcc.Graph(id='bar-chart')
    ]),
    html.Hr(),
    html.H1("Yearly Shark Attacks"),
    html.H4("Select State"),
    dropdown,
    line_graph
])

# Define callback to update the bar chart based on dropdown selection
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('state-dropdown', 'value')]
)
def update_bar_chart(selected_state):
    filtered_dff = dff[dff['State'] == selected_state]
    fig = px.bar(filtered_dff, x='Shark.common.name', y='count', color='Shark.common.name',
                 labels={'count': 'Number of Attacks', 'Shark.common.name': 'Shark Common Name'})
    fig.update_layout(title=f'Shark Attack Counts by Name in {selected_state}')
    return fig

# Define callback to update the line graph based on dropdown selection
@app.callback(
    Output('line_graph', 'figure'),
    [Input('dropdown', 'value')]
)
def update_line_chart(selected_state):
    if selected_state is None:
        filtered_df = yearly_grouped
        x_axis = filtered_df['Incident.year']
    else:
        filtered_df = yearly_df.loc[yearly_df['State'] == selected_state].sort_values(by=['Incident.year'])
        x_axis = 'Incident.year'

    new_graph = px.line(data_frame=filtered_df, x=x_axis, y='Count')
    new_graph.update_layout(title=f'Yearly Shark Attacks in {selected_state}' if selected_state else 'Yearly Shark Attacks')
    return new_graph

# run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=222)